In [1]:
import numpy as np
from dynamicSceneGenerator import DynamicSceneGenerator
from visualize import visualize_dynamic_scene, visualize_camera_pose_in_dsg, visualize_projections, visualize_bounding_boxes, visualize_distorted_bounding_boxes
from MODSIM import create_and_place_simple_legacy_camera, project_all_points_from_json, project_all_points, create_all_bbs_from_json, create_dynamic_scene_with_random_tracks 
from utils import find_path_to_next_simulation
from errorGenerator import ErrorGenerator
from datatypes.cameraRig import CameraRig
from datatypes.virtualCamera import VirtualCamera

# Might need to pip install json, json_stream, as they are not default libraries
import json
import os


In [2]:
# Find folder to save simultation trial to
simulation_folder = find_path_to_next_simulation()

In [3]:
# Generate dynamic scene with random tracks
dsg = create_dynamic_scene_with_random_tracks(6, writeToJson=True, path=simulation_folder)

In [4]:
# Visualize dynamic scene
vessels = dsg.get_vessels()
# visualize_dynamic_scene(vessels, folder_path=simulation_folder)

In [5]:
##### Static Camera #####
# Create simple legacy camera and place it in the dynamic scene
camera = create_and_place_simple_legacy_camera(dsg.get_larges_radius(), dsg.get_path_centre())
camera_rig = CameraRig(camera)
# Visualize the camera position
visualize_camera_pose_in_dsg(camera_rig, vessels, folder_path=simulation_folder)

In [5]:
###### Dynamic Camera ######
focal_length = 50*10**-3
image_bounds = (3600, 2400) # Pixels (x,y)
film_size = (36*10**-3, 24*10**-3)
px = film_size[0]/image_bounds[0]
py = film_size[1]/image_bounds[1]
principal_point = (image_bounds[0]/2,image_bounds[1]/2)
width_of_sensor = 36*10**-3 # Width of sensor
roll_vcf = 0
yaw_vcf = 0
pitch_vcf = np.pi / 20
pos_vcf = np.array([vessels[0].get_length()/2, 0, vessels[0].get_air_draft()/2])

camera = VirtualCamera(focal_length, px, py, principal_point, image_bounds)
camera.place_camera_on_vessel(pos_vcf, roll_vcf, pitch_vcf, yaw_vcf)
camera_rig = CameraRig(camera, vessels[0])
visualize_camera_pose_in_dsg(camera_rig, vessels, folder_path=simulation_folder, y_x_lim=600)

In [6]:
# NOT read from JSON
# all_projected_points = project_all_points(camera, vessels)

# Read from Json
all_projected_points = project_all_points_from_json(camera_rig, simulation_folder, writeToJson=True)

# Visualize projections
visualize_projections(all_projected_points, camera.image_bounds, show_box=True, folder_path=simulation_folder)

In [7]:
# Read projected points from JSON and create bounding boxes
all_bbs = create_all_bbs_from_json(simulation_folder, camera.image_bounds, occlusion=False, writeToJson=True)

In [9]:
# Visualize BBS
# With points in BBs
# FIX THIS WHEN RIGHT PROJECTED POINTS ARE HERE
# visualize_bounding_boxes(all_bbs, camera.image_bounds, projected_points=all_projected_points, show_projected_points=True, folder_path=simulation_folder)
# Without
visualize_bounding_boxes(all_bbs, camera.image_bounds, folder_path=simulation_folder)

In [10]:
path = 'detector_stats_config.yaml'
errorGenerator = ErrorGenerator(path)
errorBBs = errorGenerator.generate_all_error_BBs(all_bbs)

In [10]:

visualize_distorted_bounding_boxes(errorBBs, camera.image_bounds, show_original_BBS=True, original_BBs=all_bbs, folder_path=simulation_folder)
